In [736]:
import warnings
from typing import Iterable

import pandas as pd 

warnings.filterwarnings("ignore")


In [737]:
# Charger les données
df = pd.read_excel("Donnée Concours T7.xlsx", sheet_name="Feuille simplifiée")
df.head()

,LAST_NAME,DISPO,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,Annelize,Mercredi 22/05 de 9h à 12h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Antoine,Mercredi 22/05 de 9h à 12h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Benjamin,Mercredi 22/05 de 9h à 12h,Jeudi 23/05 de 12h à 15h,Vendredi 24/05 de 12h à 15h,Samedi 25/05 de 12h à 15h,Dimanche 26/05 de 12h à 15h,Lundi 27/05 de 12h à 15h,Mardi 28/05 de 12h à 15h,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cassandra,Mercredi 22/05 de 18h à 21h,Mercredi 22/05 de 18h à 21h,Jeudi 23/05 de 9h à 12h,Jeudi 23/05 de 12h à 15h,Vendredi 24/05 de 15h à 18h,Vendredi 24/05 de 18h à 21h,Samedi 25/05 de 15h à 18h,Samedi 25/05 de 18h à 21h,Dimanche 26/05 de 12h à 15h,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chloe,Dimanche 26/05 de 15h à 18h,Mercredi 22/05 de 18h à 21h,Jeudi 23/05 de 15h à 18h,Jeudi 23/05 de 18h à 21h,Vendredi 24/05 de 15h à 18h,Vendredi 24/05 de 18h à 21h,Lundi 27/05 de 15h à 18h,Lundi 27/05 de 18h à 21h,Mardi 28/05 de 15h à 18h,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [738]:
### Nettoyage des données ###

# Reformatter les colonnes
df.columns = [col.replace("Unnamed:", "dispo").replace("LAST_NAME", "nom").strip().lower() for col in df.columns] 

# Standardiser les espaces 
df = df.map(lambda x : ' '.join(x.split()).strip() if isinstance(x, str) else x)

# Corriger les créneaux de Margo, qui est disponible toute la journée du 25, pas le 22
df.loc[17] = df.loc[17].apply(lambda x : x.replace("22/05", "25/05").replace("Mercredi", "Samedi") if isinstance(x, str) else x)

# Puisqu'elle est dispo toute la journée, il faut que je rajoute le créneau de 18-21h
df.loc[17, "dispo 4"] = "Samedi 25/05 de 18h à 21h"

# Vérifier que la correction pour Margo à bien fonctionné
df.loc[17].head()

nom                            Margo
dispo       Samedi 25/05 de 9h à 12h
dispo 2    Samedi 25/05 de 12h à 15h
dispo 3    Samedi 25/05 de 15h à 18h
dispo 4    Samedi 25/05 de 18h à 21h
Name: 17, dtype: object

In [739]:
# Mettre les données sous forme d'un dictionnaire python
dispo_dict = {}
for name in df.nom.unique():

    dispos = [val for val in df[df["nom"] == name].values[0] if not pd.isna(val)][1:]
    dispo_dict[name] = dispos

# Ranger le dictionnaire pour avoir les personnes avec le moins de creneaux en premier
dispo_dict = dict(sorted(dispo_dict.items() , key=lambda item : len(item[1])))

In [740]:
# Création d'un dataframe vide avec en colonne les créneau (heures) et en index les journées
to_fill_planing = pd.DataFrame(columns=["9h à 12h" , "12h à 15h" , "15h à 18h" , "18h à 21h"], index=list(range(22, 29)))
to_fill_planing

,9h à 12h,12h à 15h,15h à 18h,18h à 21h
22,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN
28,NaN,NaN,NaN,NaN


In [741]:
placed = set() # Pour storer les personnes qu'on a placé

day : int
for day in to_fill_planing.index:

    row : pd.Series = to_fill_planing.loc[day]

    creneau : str
    for creneau in to_fill_planing.columns:
    
        name : str 
        dispos : Iterable
        for name, dispos in dispo_dict.items():
        
            if name not in placed: # Si la personne n'est pas déjà placée
                
                if any(str(day) in disp for disp in dispos): # Si la personne est dispo ce jour là
                    
                    creneau_dispo = row[row.isna()] # Filtrer sur les valeurs nulles uniquement (les creneaux dispos)
                    
                    if not creneau_dispo.empty: # Voir s'il y'a un creneau dispo
                        
                        dispo : str
                        # Voir si la personne a ce creneau de dispo aussi
                        for dispo in dispos: # Dispo de la personne

                            creneau : str
                            for creneau_ in creneau_dispo.index: # Les dispo libre du planing

                                if creneau_ in dispo and str(day) in dispo and name not in placed: 
                                    

                                    # On a un match ! 
                                    placed.add(name)
                                    to_fill_planing.loc[day, creneau_] = name
                                    break 
                        break

In [742]:
to_fill_planing

,9h à 12h,12h à 15h,15h à 18h,18h à 21h
22,Annelize,NaN,NaN,NaN
23,NaN,Benjamin,Chloe,Lorenzo
24,NaN,NaN,NaN,Fany
25,Margo,Mila,Cassandra,Mattéo
26,Nicolas,Elie,Mathis,Clément
27,NaN,Pierre,Jessy,Helene
28,Roxanne,Lucas Leverreso,Felix,Marius


In [743]:

def get_rest_des_creneaux() -> dict:
    # Récupérer le reste des créneaux disponnibles
    reste_des_creneaux = {}

    row_null_vals = to_fill_planing[to_fill_planing.isnull().any(axis=1)]

    for day in row_null_vals.index:
        crens = row_null_vals.loc[day][row_null_vals.loc[day].isnull()].index

        for cren in crens:
            reste_des_creneaux.setdefault(day, [])
            reste_des_creneaux[day].append(cren)

    return reste_des_creneaux


reste_des_creneaux = get_rest_des_creneaux()

In [744]:
# Récupérer les personnes sans créneaux

def get_not_placed() -> list:
    all_names = df["nom"].unique()
    return [nom for nom in all_names if nom not in placed]

not_placed = get_not_placed()

In [745]:
# Maintenant, implémenter une approche de switch afin de placer les dernières personnes, peut être en échangeant leurs place avec quelqu'un qui a plusieurs créneaux dispos

# Première étape; trouver une personne qui est dispo pour le creneau vide (et qui est placé). Appelons cette personne X
# Deuxième étape; voir si une des personnes pas placé peut prendre le créneau dans lequel X est placé
# Si c'est le cas, X et la personne pas placée échange leurs créneaux


creneau_rempli = set()
switched_name = set()
serie_stacked : pd.Series = to_fill_planing.stack() # Pour pouvoir récupérer la colonne et l'index d'une valeur spécifique du df

day : int 
list_creneau : list
for day, list_creneau in reste_des_creneaux.items():

    creneau_vide_ = to_fill_planing[(to_fill_planing.index == day)][list_creneau]
    
    name : str
    for name in dispo_dict:
        
        if name in placed and name not in switched_name: # on ne considère que les personnes placées et qui n'ont pas déjà switché

            cren : str
            for cren in creneau_vide_:

                if str(day) + cren not in creneau_rempli:

                    # Récupérer les dispos de la journée sur laquelle on itère pour X
                    dispos_X = [disp for disp in dispo_dict[name] if str(day) in disp]

                    for disp_ in dispos_X:

                        if cren in disp_: # On a une dispo de X pour le créneau vide dans lequel on itère ! Maintenant, il faut voir si une des personnes pas placé peut échanger sa place avec X
                            
                            index_ : int 
                            col_ : str
                            # Récupérer le placement de X
                            index_ , col_ = serie_stacked[serie_stacked == name].idxmax() # index_ est le jour, col_ est le creneau

                            name_not_placed : str
                            # Maintenant on cherche à savoir si une des personnes pas placé peut échanger son créneau avec X (donc elle est dispo là où X est placé)
                            for name_not_placed in not_placed:
                                
                                dispo_not_placed_pers = dispo_dict[name_not_placed]

                                disp_not_placed : str
                                for disp_not_placed in dispo_not_placed_pers:

                                    # Vérifier si la personne pas placé peut prendre le créneau de X
                                    if str(index_) in disp_not_placed and col_ in disp_not_placed:

                                        if name not in switched_name:

                                            if name_not_placed not in switched_name:
                                        
                                                # Si c'est le cas, on switch X et la personne pas placée 

                                                # Placer X dans le créneau vide
                                                to_fill_planing.loc[day , cren] = name 

                                                # Placer la personne pas placée dans le créneau de X 
                                                to_fill_planing.loc[index_ , col_] = name_not_placed 

                                                # Faire en sorte qu'on ne reitère pas sur une personne qu'on vient de placer
                                                placed.add(name_not_placed)

                                                creneau_rempli.add(str(day) + cren)
                                                switched_name.add(name)
                                                switched_name.add(name_not_placed)
                                                break 

                                                
                                    
to_fill_planing              

,9h à 12h,12h à 15h,15h à 18h,18h à 21h
22,Annelize,Jessy,Lucas Leverreso,Chloe
23,Cassandra,Wil,Jacquier,Mathilde
24,Lorenzo,Benjamin,NaN,Fany
25,Margo,Mila,Maria,Mattéo
26,Nicolas,Elie,Mathis,Clément
27,NaN,Pierre,Manon,Helene
28,Roxanne,Lucas Point,Felix,Marius


In [746]:
reste_des_creneaux = get_rest_des_creneaux()
not_placed = get_not_placed()

reste_des_creneaux

{24: ['15h à 18h'], 27: ['9h à 12h']}

In [747]:
not_placed

['Antoine', 'Colombe']

In [748]:
to_fill_planing.to_excel("planning_complet.xlsx")